## Import the required libraries


In [120]:

import sys
import os
import numpy as np
import pandas as pd
from scipy.fft import fft
from scipy.signal import ShortTimeFFT,butter,hilbert,sosfiltfilt,medfilt
from scipy.signal.windows import gaussian
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.ticker import ScalarFormatter
import tkinter as tk
from tkinter import filedialog
from sklearn.decomposition import PCA
import os
from io import StringIO
from matplotlib import gridspec
%matplotlib qt


## Define functions


In [121]:
# Prompt the user to select a folder
def select_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    directory = filedialog.askdirectory()  # Open the file selection dialog
    return directory

# Prompt the user to open a file 
def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename()  # Open the file selection dialog
    return file_path

def choose_option(option1, option2, option3, option4):
    result = {}

    def select(choice):
        result['value'] = choice
        root.destroy()

    root = tk.Tk()
    root.title("Choose the type of visual stim")

    tk.Label(root, text="Please choose the type of visual stim:").pack(pady=10)
    tk.Button(root, text=option1, width=12, command=lambda: select(option1)).pack(side='left', padx=10, pady=10)
    tk.Button(root, text=option2, width=12, command=lambda: select(option2)).pack(side='left', padx=10, pady=10)
    tk.Button(root, text=option3, width=12, command=lambda: select(option3)).pack(side='left', padx=10, pady=10)
    tk.Button(root, text=option4, width=12, command=lambda: select(option4)).pack(side='left', padx=10, pady=10)

    # Manual event loop, blocks until window is destroyed
    while not result.get('value'):
        root.update()

    return result['value']

# Function to remove parentheses characters from a line
def remove_parentheses_chars(line):
    # Remove only '(' and ')' characters
    return line.replace('(', '').replace(')', '')
def clean_csv(filename):
    with open(filename, 'r') as f:
        lines = [remove_parentheses_chars(line) for line in f]
    # Join lines and create a file-like object
        cleaned = StringIO(''.join(lines))
        return cleaned

# Butterworth filter to remove high frequency noise
def butter_noncausal(signal, fs, cutoff_freq=1, order=4):
    sos = butter(order, cutoff_freq/(fs/2), btype='low', output='sos')  # 50 Hz cutoff frequency
    return sosfiltfilt(sos, signal)   

def interpolate_nans(arr):
    nans = np.isnan(arr)
    x = np.arange(len(arr))
    arr[nans] = np.interp(x[nans], x[~nans], arr[~nans])
    return arr

def rotation_matrix(angle_rad):
    return np.array([[np.cos(angle_rad), -np.sin(angle_rad)],
                     [np.sin(angle_rad), np.cos(angle_rad)]])


pca = PCA(n_components=2)

def vector_to_rgb(angle, absolute): ##Got it from https://stackoverflow.com/questions/19576495/color-matplotlib-quiver-field-according-to-magnitude-and-direction
    global max_abs

    # normalize angle
    angle = angle % (2 * np.pi)
    if angle < 0:
        angle += 2 * np.pi

    # return matplotlib.colors.hsv_to_rgb((angle / 2 / np.pi, 
    #                                      absolute / max_abs, 
    #                                      absolute / max_abs))
    return matplotlib.colors.hsv_to_rgb((angle / 2 / np.pi, 
                                         1, 
                                         1))
def plot_angle_distribution(angle, ax_polar, num_bins=18):
    """
    Plots a normalized polar histogram of angles.

    Parameters:
        angle (np.ndarray): array of saccade angles in radians
        ax_polar (matplotlib.axes._subplots.PolarAxesSubplot): the polar subplot to draw on
        num_bins (int): number of histogram bins
    """
    angle_2pi = np.where(angle < 0, angle + 2 * np.pi, angle)
    counts, bin_edges = np.histogram(angle_2pi, bins=num_bins, range=(0, 2 * np.pi))
    counts = counts / np.size(angle_2pi)  # Normalize
    width = np.diff(bin_edges)

    bars = ax_polar.bar(bin_edges[:-1], counts, width=width, align='edge', color='b', alpha=0.5, edgecolor='k')
    ax_polar.set_title("Normalized angle distribution")
    ax_polar.set_yticklabels([])

def plot_linear_histogram(angles, ax, num_bins=18):
    ang_deg = np.degrees(angles)
    ang_deg = np.mod(ang_deg, 360)
    counts, bins = np.histogram(ang_deg, bins=num_bins, range=(0, 360))
    counts = counts / ang_deg.size
    ax.bar(bins[:-1], counts, width=np.diff(bins), color="b", alpha=0.5, edgecolor="k")
    ax.set_xlabel("Angle (deg)")
    ax.set_ylabel("Normalised count")
    ax.set_title("Linear angle histogram")    

def analyze_eye_saccades(
    marker1_x, marker1_y, marker2_x, marker2_y,
    gaze_x, gaze_y,
    calibration_factor,
    go_frame, go_direction,
    blink_velocity_threshold, 
    saccade_threshold,
    blink_detection=0,
    vd_axis_lx=None, vd_axis_ly=None, vd_axis_rx=None, vd_axis_ry=None,
):
    eye_origin = np.column_stack((
        (marker1_x + marker2_x) / 2,
        (marker1_y + marker2_y) / 2
    ))
    eye_angle = np.arctan2(marker2_y - marker1_y, marker2_x - marker1_x)

    eye_camera = np.column_stack((gaze_x - eye_origin[:, 0], gaze_y - eye_origin[:, 1]))
    eye_camera[:, 0] = medfilt(eye_camera[:, 0], kernel_size=3)
    eye_camera[:, 1] = medfilt(eye_camera[:, 1], kernel_size=3)

    eye_camera = eye_camera / calibration_factor
    eye_camera_diff = np.zeros_like(eye_camera)
    eye_camera_diff[:, 0] = np.ediff1d(eye_camera[:, 0], to_begin=0)
    eye_camera_diff[:, 1] = np.ediff1d(eye_camera[:, 1], to_begin=0)
    saccade_indices = np.where(np.linalg.norm(eye_camera_diff, axis=1) >= saccade_threshold)[0]

    if blink_detection:
        vd_axis_left = np.vstack([vd_axis_lx, vd_axis_ly]).T
        vd_axis_right = np.vstack([vd_axis_rx, vd_axis_ry]).T
        vd_axis_d = np.linalg.norm(vd_axis_right - vd_axis_left, axis=1)
        vd_axis_vel = np.gradient(vd_axis_d)
        blink_indices = np.where(np.abs(vd_axis_vel) > blink_velocity_threshold)
        saccade_indices = saccade_indices[~np.isin(saccade_indices, blink_indices)]

    stim_left = go_frame[np.where(go_direction < 0)[0]]
    stim_right = go_frame[np.where(go_direction > 0)[0]]

    return eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right


def plot_eye_saccades(
    eye_camera, eye_camera_diff, saccade_indices,
    stim_left, stim_right,
    session_path,
    stim_type='None',
    eye_name='Eye',
    stim_direction='None',
    plot_window=np.arange(0, 30, 1)
):
    
    session_name = os.path.basename(session_path.rstrip("/\\"))

# 1️⃣  GLOBAL LIMITS  (from ALL saccade points)
    x_all = eye_camera[saccade_indices, 0]
    y_all = eye_camera[saccade_indices, 1]

    pad  = 0.1  # 5 % padding around data
    rngX = x_all.max() - x_all.min()
    rngY = y_all.max() - y_all.min()

    X_LIM = (x_all.min() - pad*rngX, x_all.max() + pad*rngX)
    Y_LIM = (y_all.min() - pad*rngY, y_all.max() + pad*rngY)


    max_abs = np.max(np.abs(eye_camera_diff))
    angle = np.arctan2(
        eye_camera_diff[saccade_indices, 1],
        eye_camera_diff[saccade_indices, 0]
    )
    colors = np.array([vector_to_rgb(a, max_abs) for a in angle])

    # ---- figure layout: 1 col (quiver)  + 2 stacked sub‑axes on right --------
    fig = plt.figure(figsize=(14, 6))
    gs = gridspec.GridSpec(2, 2, width_ratios=[3, 2])
    ax_quiver = fig.add_subplot(gs[:, 0])          # spans both rows, left column
    ax_polar  = fig.add_subplot(gs[0, 1], polar=True)
    ax_linear = fig.add_subplot(gs[1, 1])


    # --- All saccades: Quiver + Polar + Colorwheel
    ax_quiver.set_xlim(*X_LIM)
    ax_quiver.set_ylim(*Y_LIM)
    ax_quiver.set_xlabel('X Position ($\\degree$)')
    ax_quiver.set_ylabel('Y Position ($\\degree$)')
    ax_quiver.set_title(f"{session_name}\nAll saccades {eye_name} (stim: {stim_type})")


    x = eye_camera[saccade_indices, 0]
    y = eye_camera[saccade_indices, 1]
    u = eye_camera_diff[saccade_indices, 0]
    v = eye_camera_diff[saccade_indices, 1]
    ax_quiver.quiver(x, y, u, v, angles='xy', scale_units='xy', scale=1, color=colors, alpha=0.5)

    pca.fit(eye_camera_diff[saccade_indices] / np.linalg.norm(eye_camera_diff[saccade_indices], axis=1, keepdims=True))
    components = pca.components_
    evs = pca.explained_variance_ratio_
    origin = np.mean(eye_camera[saccade_indices], axis=0)
    for i, (comp, var) in enumerate(zip(components, evs)):
        vector = comp * 10 * np.sqrt(var)
        ax_quiver.arrow(origin[0], origin[1], vector[0], vector[1],
                        color=['k', 'b'][i], width=0.1, label=f'PC{i + 1} ({var:.2f} var)')
    ax_quiver.legend()

    # --- Polar histogram and colorwheel ---
    plot_angle_distribution(angle, ax_polar)
    plot_linear_histogram(angle, ax_linear)

    # --- Per-stimulus saccade plots
    if stim_direction != 'None':
        direction_map = {
            'Any': ['Left/Down', 'Right/Up'],
            'LR': ['Left', 'Right'],
            'UD': ['Down', 'Up']
        }
        labels = direction_map.get(stim_direction, ['Left', 'Right'])
        stim_sets = [stim_left, stim_right]

        for stim_set, label in zip(stim_sets, labels):
            angles_to_plot = []
            
            # Create figure and manually set subplot types

            fig = plt.figure(figsize=(14, 6))
            gs = gridspec.GridSpec(2, 2, width_ratios=[3, 2])
            ax_q = fig.add_subplot(gs[:, 0])          # spans both rows, left column
            ax_p = fig.add_subplot(gs[0, 1], polar=True)
            ax_l = fig.add_subplot(gs[1, 1])
            

            for f in stim_set:
                saccades_in_window = saccade_indices[
                    (saccade_indices >= f + plot_window[0]) & (saccade_indices <= f + plot_window[-1])
                ]
                x = eye_camera[saccades_in_window, 0]
                y = eye_camera[saccades_in_window, 1]
                u = eye_camera_diff[saccades_in_window, 0]
                v = eye_camera_diff[saccades_in_window, 1]
                angle = np.arctan2(v, u)
                angles_to_plot.append(angle)
                colors = np.array([vector_to_rgb(a, max_abs) for a in angle])
                ax_q.quiver(x, y, u, v, angles='xy', scale_units='xy', scale=1, color=colors, alpha=0.5)
                ax_q.set_title(f"{session_name}\nSaccades for {label} stimulus ({eye_name})")
                ax_q.set_xlim(*X_LIM)
                ax_q.set_ylim(*Y_LIM)
                ax_q.set_xlabel('X Position ($\\degree$)')
                ax_q.set_ylabel('Y Position ($\\degree$)')

            if angles_to_plot:
                combined = np.concatenate(angles_to_plot)
                # --- Polar histogram and colorwheel ---
                plot_angle_distribution(combined, ax_p)
                plot_linear_histogram(combined, ax_l)

## Define parameters

In [122]:
cal = 3.76  # Calibration factor for the pixels to degrees
ttl_freq = 60  # TTL frequency in Hz

# Parameters for nlink and saccade detection
blink_thresh= 10
saccade_thresh=1.5

#folder_path = select_folder() #this won't work if you're running jupyter lab in browser, so hard coding
folder_path = r"X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T13_02_29\\" # Path to the folder containing the data files
stim_type = choose_option("None","LR","UD","Interleaved")


## Extract relevant files from filepath

In [123]:
#initialize variables to store file paths
IMU_file = None
camera_file = None
go_file = None
ellipse_center_XY_R_file = None
origin_of_eye_coordinate_R_file = None
vdaxis_R_file = None
ellipse_center_XY_L_file = None
origin_of_eye_coordinate_L_file = None
vdaxis_L_file = None
blink_detection_r = 0
blink_detection_l = 0

print(f"Scanning folder: {folder_path}")
print(f"Found {len(os.listdir(folder_path))} files")

# Scan the folder for specific files
for f in os.listdir(folder_path):
    f_lower = f.lower()
    full_path = os.path.join(folder_path, f)

    if 'imu' in f_lower:
        IMU_file = full_path
    elif 'camera' in f_lower:
        camera_file = full_path
    elif 'go' in f_lower:
        go_file = full_path
    elif 'ellipse_center_xy_r' in f_lower:
        ellipse_center_XY_R_file = full_path
    elif 'origin_of_eyecoordinate_r' in f_lower:
        origin_of_eye_coordinate_R_file = full_path
    elif 'vdaxis_r' in f_lower:
        vdaxis_R_file = full_path
        blink_detection_r = 1
    elif 'ellipse_center_xy_l' in f_lower:
        ellipse_center_XY_L_file = full_path
    elif 'origin_of_eyecoordinate_l' in f_lower:
        origin_of_eye_coordinate_L_file = full_path
    elif 'vdaxis_l' in f_lower:
        vdaxis_L_file = full_path
        blink_detection_l = 1

# Check if all required files are found
if IMU_file == None and camera_file == None:
    raise ValueError('IMU and Camera files not found in the selected folder.')
if IMU_file == None:
    raise ValueError('IMU file not found in the selected folder.')
if camera_file == None:
    raise ValueError('Camera file not found in the selected folder.')
if go_file == None:
    raise ValueError('go file not found in the selected folder.')
if ellipse_center_XY_R_file == None:
    raise ValueError('ellipse_center_XY_R file not found in the selected folder.')
if origin_of_eye_coordinate_R_file == None: 
    raise ValueError('origin_of_eye_coordinate_R file not found in the selected folder.')
if ellipse_center_XY_L_file == None:
    raise ValueError('ellipse_center_XY_L file not found in the selected folder.')  
if origin_of_eye_coordinate_L_file == None:
    raise ValueError('origin_of_eye_coordinate_L file not found in the selected folder.')
if vdaxis_R_file == None:
    print("Warning: Right VD Axis file is not present. Saccade detection might be bad especially for rats!!:")
if vdaxis_L_file == None:
    print("Warning: Left VD Axis file is not present. Saccade detection might be bad especially for rats!!")



Scanning folder: X:\Experimental_Data\EyeHeadCoupling_RatTS_server\TSh01_Paris_server\Tsh001_2025-06-11T13_02_29\\
Found 23 files


## Extract stim and eye position data (Right, Left only if it exists)

In [124]:
## Read the camera data and map between camera TTL and Bonsai TTLs
camera_data = np.genfromtxt(camera_file, delimiter=',', skip_header=1, dtype=np.float64)
[bonsai_frame, bonsai_time] = camera_data[:, 0], camera_data[:, 1]
bonsai_frame = bonsai_frame.astype(int)  # Convert bonsai_frame to integer type


### Read the go file for the start of stim in the trial 
new_go_data_format=0
go_data = np.genfromtxt(clean_csv(go_file), delimiter=',', skip_header=1, dtype=np.float64)
if go_data.shape[1]>3:
    new_go_data_format = 1
    [go_frame, go_time, go_direction_x,go_direction_y] = go_data[:, 0], go_data[:, 1], go_data[:, 2], go_data[:,3]
else:
    [go_frame, go_time, go_direction] = go_data[:, 0], go_data[:, 1], go_data[:, 2]
go_frame = go_frame.astype(int)  # Convert go_frame to integer type

### Read the ellipse center XY R file for the right eye 
ellipse_center_XY_R_data = np.genfromtxt(clean_csv(ellipse_center_XY_R_file), delimiter=',', skip_header=1, dtype=np.float64)
[eye_frame_r,eye_timestamp_r,eye_rx,eye_ry] = ellipse_center_XY_R_data[:, 0], ellipse_center_XY_R_data[:, 1], ellipse_center_XY_R_data[:, 2], ellipse_center_XY_R_data[:, 3]
eye_frame_r = eye_frame_r.astype(int)  # Convert eye_frame_r to integer type
eye_rx = interpolate_nans(eye_rx)  # Interpolate NaN values in eye_rx
eye_ry = interpolate_nans(eye_ry)  # Interpolate NaN values in eye_ry

### Read the origin of eye coordinate R file for the right eye 
origin_of_eye_coordinate_R_data = np.genfromtxt(clean_csv(origin_of_eye_coordinate_R_file), delimiter=',', skip_header=1, dtype=np.float64)
[origin_frame_r,o_ts,l_rx,l_ry,r_rx,r_ry] = origin_of_eye_coordinate_R_data[:, 0], origin_of_eye_coordinate_R_data[:, 1], origin_of_eye_coordinate_R_data[:, 2], origin_of_eye_coordinate_R_data[:, 3], origin_of_eye_coordinate_R_data[:, 4], origin_of_eye_coordinate_R_data[:, 5]
origin_frame_r = origin_frame_r.astype(int)  # Convert origin_frame_r to integer type
l_rx = interpolate_nans(l_rx)  # Interpolate NaN values in l_rx
r_rx = interpolate_nans(r_rx)  # Interpolate NaN values in r_rx 
l_ry = interpolate_nans(l_ry)  # Interpolate NaN values in l_ry
r_ry = interpolate_nans(r_ry)  # Interpolate NaN values in r_ry

## Read the vertical (VD) axis data for right eye - this is used for blink detection
if (blink_detection_r == 1):
    vdaxis_R_data = np.genfromtxt(clean_csv(vdaxis_R_file),delimiter=',',skip_header=1,dtype=np.float64)
    [vd_frame_r,vd_r_ts,vd_r_lx,vd_r_ly,vd_r_rx,vd_r_ry] = vdaxis_R_data[:,0],vdaxis_R_data[:,1],vdaxis_R_data[:,2],vdaxis_R_data[:,3],vdaxis_R_data[:,4],vdaxis_R_data[:,5]
    vd_frame_r = vd_frame_r.astype(int)
    # Interpolate NaN values
    vd_r_lx = interpolate_nans(vd_r_lx)
    vd_r_ly = interpolate_nans(vd_r_ly)
    vd_r_rx = interpolate_nans(vd_r_rx)
    vd_r_ry = interpolate_nans(vd_r_ry)

# Check if the left eye data files exist and read them if they do
ellipse_center_XY_L_file = ellipse_center_XY_L_file if 'ellipse_center_xy_l' in locals() else None
origin_of_eye_coordinate_L_file = origin_of_eye_coordinate_L_file if 'origin_of_eyecoordinate_l' in locals() else None
vdaxis_L_file = vdaxis_L_file if 'vdaxis_l' in locals() else None
# Initialize variables for left eye data
eye_frame_l = None
eye_timestamp_l = None
eye_lx = None
eye_ly = None
origin_frame_l = None
l_lx = None
r_lx = None
l_ly = None

r_ly = None
vd_frame_l = None
vd_l_ts = None
vd_l_lx = None
vd_l_ly = None
vd_l_rx = None
vd_l_ry = None
# Attempt to read the left eye data files if they exist
if ellipse_center_XY_L_file and origin_of_eye_coordinate_L_file:
    print("Reading left eye data files...")
    blink_detection_l = 1  # Set blink detection for left eye to 1 if files are present
    try:
        ### Read the ellipse center XY L file for the left eye using numpy
        ellipse_center_XY_L_data = np.genfromtxt(clean_csv(ellipse_center_XY_L_file), delimiter=',', skip_header=1, dtype=np.float64)
        [eye_frame_l,eye_timestamp_l,eye_lx,eye_ly] = ellipse_center_XY_L_data[:, 0], ellipse_center_XY_L_data[:, 1], ellipse_center_XY_L_data[:, 2], ellipse_center_XY_L_data[:, 3]
        eye_frame_l = eye_frame_l.astype(int)  # Convert eye_frame_l to integer type
        eye_lx = interpolate_nans(eye_lx)  # Interpolate NaN values in eye_lx
        eye_ly = interpolate_nans(eye_ly)  # Interpolate NaN values in eye_ly

        ### Read the origin of eye coordinate L file for the left eye using numpy       
        origin_of_eye_coordinate_L_data = np.genfromtxt(clean_csv(origin_of_eye_coordinate_L_file), delimiter=',', skip_header=1, dtype=np.float64)
        [origin_frame_l,o_ts_l,l_lx,l_ly,r_lx,r_ly] = origin_of_eye_coordinate_L_data[:, 0], origin_of_eye_coordinate_L_data[:, 1], origin_of_eye_coordinate_L_data[:, 2], origin_of_eye_coordinate_L_data[:, 3], origin_of_eye_coordinate_L_data[:, 4], origin_of_eye_coordinate_L_data[:, 5]
        origin_frame_l = origin_frame_l.astype(int)  # Convert origin_frame_l to integer type
        l_lx = interpolate_nans(l_lx)  # Interpolate NaN values in l_lx
        r_lx = interpolate_nans(r_lx)  # Interpolate NaN values in r_lx 
        l_ly = interpolate_nans(l_ly)  # Interpolate NaN values in l_ly
        r_ly = interpolate_nans(r_ly)  # Interpolate NaN values in r_ly
    except Exception as e:
        print(f"Error reading left eye data files: {e}")
else:
    print("Warning: Left eye data files are not present. Saccade detection might be bad especially for rats!!")

## Read the VD axis data for left eye
if (blink_detection_l==1):
    try:
        vdaxis_L_data = np.genfromtxt(clean_csv(vdaxis_L_file), delimiter=',', skip_header=1, dtype=np.float64)
        [vd_frame_l, vd_l_ts, vd_l_lx, vd_l_ly, vd_l_rx, vd_l_ry] = vdaxis_L_data[:, 0], vdaxis_L_data[:, 1], vdaxis_L_data[:, 2], vdaxis_L_data[:, 3], vdaxis_L_data[:, 4], vdaxis_L_data[:, 5]
        vd_frame_l = vd_frame_l.astype(int)

        # Interpolate NaN values
        vd_l_lx = interpolate_nans(vd_l_lx)
        vd_l_ly = interpolate_nans(vd_l_ly)
        vd_l_rx = interpolate_nans(vd_l_rx)
        vd_l_ry = interpolate_nans(vd_l_ry)
    except Exception as e:
        print(f"Error reading left VD axis data file: {e}") 

        
### Read the IMU data for the accelerometer and gyroscope
imu_data = np.genfromtxt(IMU_file, delimiter=',', skip_header=1, dtype=np.float64)
[imu_time,a_x,a_y,a_z,g_x,g_y,g_z,m_x,m_y,m_z] = imu_data[:, 0], imu_data[:, 1], imu_data[:, 2], imu_data[:, 3], imu_data[:, 4], imu_data[:, 5], imu_data[:, 6], imu_data[:, 7], imu_data[:, 8], imu_data[:, 9]
imu_time = imu_time.astype(np.float64)  # Ensure imu_time is in float64 format
# Interpolate NaN values in IMU data
a_x = interpolate_nans(a_x)
a_y = interpolate_nans(a_y)
a_z = interpolate_nans(a_z)
g_x = interpolate_nans(g_x)
g_y = interpolate_nans(g_y)
g_z = interpolate_nans(g_z)
m_x = interpolate_nans(m_x)
m_y = interpolate_nans(m_y)
m_z = interpolate_nans(m_z)


Error reading left VD axis data file: expected str, bytes or os.PathLike object, not NoneType


## Analyze and plot saccade statistics

In [ ]:
if (new_go_data_format):
    if (stim_type=='None'):
            eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right= analyze_eye_saccades(
            l_rx, l_ry, r_rx, r_ry,
            eye_rx, eye_ry,
            cal,
            go_frame, go_direction_x,
            blink_thresh,
            saccade_thresh,
            blink_detection=blink_detection_r,
            vd_axis_lx=vd_r_lx,
            vd_axis_ly=vd_r_ly,
            vd_axis_rx=vd_r_rx,
            vd_axis_ry = vd_r_ry,
            )
            plot_eye_saccades(eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right,
                              folder_path,stim_type, eye_name='Right Eye', stim_direction = 'None')



    if (stim_type=='Interleaved'):
            eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right= analyze_eye_saccades(
            l_rx, l_ry, r_rx, r_ry,
            eye_rx, eye_ry,
            cal,
            go_frame, go_direction_x,
            blink_thresh,
            saccade_thresh,
            blink_detection=blink_detection_r,
            vd_axis_lx=vd_r_lx,
            vd_axis_ly=vd_r_ly,
            vd_axis_rx=vd_r_rx,
            vd_axis_ry = vd_r_ry,
            )
            plot_eye_saccades(eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right,
                                folder_path, stim_type, eye_name='Right Eye', stim_direction = 'LR')

            
            eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right= analyze_eye_saccades(
            l_rx, l_ry, r_rx, r_ry,
            eye_rx, eye_ry,
            cal,
            go_frame, go_direction_y,
            blink_thresh,
            saccade_thresh,
            blink_detection=blink_detection_r,
            vd_axis_lx=vd_r_lx,
            vd_axis_ly=vd_r_ly,
            vd_axis_rx=vd_r_rx,
            vd_axis_ry = vd_r_ry,
            )
            plot_eye_saccades(eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right,
                              folder_path, stim_type, eye_name='Right Eye', stim_direction = 'UD')
            
            plt.close()  #close duplicate all saccade plot

else:    
        eye_camera, eye_camera_diff, saccade_indices, stim_left, stim_right= analyze_eye_saccades(
        l_rx, l_ry, r_rx, r_ry,
        eye_rx, eye_ry,
        cal,
        go_frame, go_direction,
        blink_thresh,
        saccade_thresh,
        blink_detection=blink_detection_r,
        vd_axis_lx=vd_r_lx,
        vd_axis_ly=vd_r_ly,
        vd_axis_rx=vd_r_rx,
        vd_axis_ry = vd_r_ry,
        )
        
        plot_eye_saccades(eye_camera,eye_camera_diff, saccade_indices, stim_left, stim_right, 
                          folder_path,stim_type, eye_name='Right Eye', stim_direction=stim_type)
